In [1]:
import gym
import numpy as np
import pandas as pd
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from StockTrade import StockTrade

In [12]:
env = StockTrade(stock_quantity=8)

In [13]:
env.np_data.shape

(1535, 40)

In [2]:
env = gym.make('Pendulum-v0')

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG('MlpPolicy', env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=10000, log_interval=10)
model.save("ddpg_pendulum")
env = model.get_env()

del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_pendulum")

obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.41e+03 |
| time/              |           |
|    episodes        | 10        |
|    fps             | 59        |
|    time_elapsed    | 33        |
|    total timesteps | 2000      |
| train/             |           |
|    actor_loss      | 57.8      |
|    critic_loss     | 15        |
|    learning_rate   | 0.001     |
|    n_updates       | 1800      |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -1.2e+03 |
| time/              |          |
|    episodes        | 20       |
|    fps             | 56       |
|    time_elapsed    | 70       |
|    total timesteps | 4000     |
| train/             |          |
|    actor_loss      | 94.2     |
|    c

In [1]:
obs = env.reset()
dones = False
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
env.close()

NameError: name 'env' is not defined

In [1]:
from StockDataPreProcessor import StockDataPreProcessor

In [2]:
prepro = StockDataPreProcessor()
pds = prepro.get_data()

In [4]:
pds


,Date,Open_x,High_x,Low_x,Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,...,Open_y,High_y,Low_y,Close_y,Volume_y,Open,High,Low,Close,Volume
0,2005-02-25,17.711,18.255,17.593,18.255,128722,9.788,10.0150,9.6180,9.986,...,5.96,6.1800,5.95,6.09,988382,37.506,38.0430,37.303,37.876,2900762
1,2005-02-28,18.255,18.255,17.957,18.191,118097,9.873,10.0430,9.7980,10.043,...,6.06,6.1500,5.81,5.82,1720101,37.859,38.5150,37.244,38.338,4261718
2,2005-03-01,18.250,18.932,18.230,18.791,151401,9.949,10.5340,9.9490,10.420,...,5.89,6.1400,5.86,6.12,1408462,38.314,38.4490,37.681,37.825,3387671
3,2005-03-02,18.791,19.063,18.580,18.605,136894,10.383,10.4770,10.2880,10.288,...,6.03,6.1700,6.00,6.06,1436016,37.724,38.0200,37.572,37.842,3124737
4,2005-03-03,18.605,18.962,18.605,18.892,63341,10.373,10.8640,10.2510,10.609,...,6.08,6.1400,6.00,6.06,824857,37.893,37.9770,37.641,37.791,1917217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,2017-11-06,30.950,31.500,30.950,31.300,237923,11.810,12.1399,11.7856,12.040,...,11.96,12.0216,11.69,11.79,966546,66.080,66.9300,66.080,66.550,1925022
3196,2017-11-07,31.500,31.950,31.250,31.750,421953,12.030,12.1100,11.3300,11.510,...,11.80,11.8100,11.61,11.81,1518445,66.500,66.8300,65.450,65.520,1489473
3197,2017-11-08,31.100,34.575,31.100,34.250,1304033,11.510,11.6572,11.2000,11.300,...,11.31,11.9900,10.12,11.86,4096736,65.620,65.9400,65.320,65.600,1377608
3198,2017-11-09,33.750,34.200,31.550,31.700,1103740,11.300,11.6700,11.2600,11.560,...,11.63,11.8100,11.11,11.43,1768651,65.440,68.0575,65.320,67.280,3915921


In [ ]:
pd.merge(pds[0],pds[1], how='inner', on='Date')


In [ ]:
pds[0].set_index('Date')
pds[1].set_index('Date')
pd.merge(pds[0],pds[1], how='inner')


In [ ]:
np.zeros(5)

,Date,Open_x,High_x,Low_x,Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Volume_y
0,2005-02-25,33.137,33.912,33.037,33.844,162215,0.7600,0.8000,0.7600,0.8000,5690
1,2005-02-28,33.031,33.569,32.925,33.286,147819,0.7800,0.7800,0.6500,0.7200,19170
2,2005-03-01,28.629,29.459,25.944,29.459,5620443,0.6500,0.7300,0.6500,0.7200,20100
3,2005-03-02,30.057,30.216,28.624,29.141,1653802,0.6700,0.6700,0.6700,0.6700,800
4,2005-03-03,29.150,29.289,28.650,29.223,468768,0.6700,0.7400,0.6700,0.7200,4000
...,...,...,...,...,...,...,...,...,...,...,...
3187,2017-11-06,79.473,79.542,78.288,78.348,109489,0.5750,0.5995,0.5555,0.5600,192902
3188,2017-11-07,78.596,78.975,77.737,78.606,134343,0.5770,0.5789,0.5571,0.5785,57529
3189,2017-11-08,78.119,78.298,76.874,78.278,87587,0.5736,0.6400,0.5700,0.5727,110618
3190,2017-11-09,77.402,78.547,76.466,78.168,115789,0.5797,0.6197,0.5721,0.5850,173762


In [10]:
pds[0].set_index('Date')
pds[1].set_index('Date')
pd.merge(pds[0],pds[1], how='inner')


,Date,Open,High,Low,Close,Volume,OpenInt


In [14]:
np.zeros(5)

array([0., 0., 0., 0., 0.])